In [ ]:
import torch
import numpy as np

In [ ]:
 #Basic Initialization

#Code:
 #setting path in colab
 #mount the drive first
from google.colab import drive
drive.mount('/content/drive/', force_remount= True)
import os

#set the working directory 

root_dir = "/content/drive/My Drive/"

#choose the project folder
project_folder = "CIS680HW/HW4b/"

#define a function to create and set the working directory
def create_and_set_working_directory(project_folder):
  #check if the project folder exists. if not, make one.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + 'did not existed and was created.')
  
  #change the OS path to project folder as working directory
  os.chdir(root_dir + project_folder)

  #create a test file in the working directory and see if it shows up at the right place
  !touch 'new_file_test.txt'
  print('working directory' + root_dir + project_folder + \
        "empty text file created. You can also run !pwd command to confirm working directory.")

create_and_set_working_directory(project_folder)

In [ ]:
cd /content/drive/MyDrive/CIS680HW/HW4b

/content


In [ ]:
from BoxHead import *
from utils import *
from dataset import BuildDataLoader, BuildDataset
from pretrained_models import pretrained_models_680
import torch.optim as optim
from torchvision.models.detection.image_list import ImageList
import matplotlib.pyplot as plt
from torchvision import transforms
import matplotlib.patches as patches

In [ ]:
imgs_path = root_dir + 'CIS680HW' + '/HW3/data/hw3_mycocodata_img_comp_zlib.h5'
masks_path = root_dir + 'CIS680HW'+'/HW3/data/hw3_mycocodata_mask_comp_zlib.h5'
labels_path = root_dir + 'CIS680HW'+'/HW3/data/hw3_mycocodata_labels_comp_zlib.npy'
bboxes_path = root_dir + 'CIS680HW'+'/HW3/data/hw3_mycocodata_bboxes_comp_zlib.npy'
paths = [imgs_path, masks_path, labels_path, bboxes_path]
# load the data into data.Dataset
dataset = BuildDataset(paths)

In [ ]:
# set 20% of the dataset as the training data
full_size = len(dataset)
train_size = int(full_size * 0.8)
test_size = full_size - train_size
torch.random.manual_seed(1)    
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 1
train_build_loader = BuildDataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
train_loader = train_build_loader.loader()
test_build_loader = BuildDataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = test_build_loader.loader()

In [ ]:
pretrained_path='models/checkpoint680.pth'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
backbone, rpn = pretrained_models_680(pretrained_path, eval=True)

# Load your model here. If you use different parameters for the initialization you can change the following code
# accordingly
keep_topK=200
boxHead=BoxHead()
boxHead=boxHead.to(device)
optimizer = optim.SGD(boxHead.parameters(),lr = 0.01/batch_size,weight_decay=1.0e-4,momentum=0.90)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
annotations = ["vehicle", "person", "animal"]

for i, batch in enumerate(test_loader,0):
  images = batch['images'].to(device)
  gt_labels = batch['labels']
  boxes = batch['bboxes']
  backout = backbone(images)

  # The RPN implementation takes as first argument the following image list
  im_lis = ImageList(images, [(800, 1088)]*images.shape[0])
  # Then we pass the image list and the backbone output through the rpn
  rpnout = rpn(im_lis, backout)
  #pdb.set_trace()
  #The final output is
  # A list of proposal tensors: list:len(bz){(keep_topK,4)}
  proposals=[proposal[0:keep_topK,:] for proposal in rpnout[0]]
  fpn_feat_list= list(backout.values())
  feature_vectors=boxHead.MultiScaleRoiAlign(fpn_feat_list.copy(),proposals.copy())
  class_logits,box_preds=boxHead(feature_vectors)
  
  class_gt,class_gt_box = boxHead.create_ground_truth(proposals,gt_labels,boxes)
  # print(boxes)
  # print(class_gt)
  img = images[0]
  img = transforms.functional.normalize(img,
                  [-0.485/0.229, -0.456/0.224, -0.406/0.225],
                  [1/0.229, 1/0.224, 1/0.225], inplace=False)
  img = img.permute(1,2,0).cpu().numpy()
  plt.figure()
  fig, ax = plt.subplots(1,1)
  ax.imshow(img)
  for coords, labels in zip(boxes, gt_labels):
    for c, l in zip(coords, labels):
      cmap = ""
      col = ''
      if l == 1:
        cmap = "jet"
        col = 'b'
      elif l == 2:
        col = 'g'
        cmap = "ocean"
      else:
        col = 'r'
        cmap = "prism"
      
      rect=patches.Rectangle((c[0],c[1]),c[2]-c[0],c[3]-c[1],fill=False,color=col)
      ax.add_patch(rect)
      ax.annotate(annotations[l - 1], (c[0] + 40, c[1] - 50), color=col, weight='bold', 
              fontsize=14, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5,edgecolor="white"))
  
  class_gt_box = torch.Tensor(class_gt_box)
  proposal_flatten = proposals.copy()
  proposal_flatten = torch.stack(proposal_flatten,dim=0)
  proposal_flatten = proposal_flatten.reshape((-1, proposal_flatten.shape[2]))
  proposals_decoded = output_decoding(class_gt_box,proposal_flatten, device=device)

  for c, l in zip(proposal_flatten[class_gt>0], class_gt[class_gt>0]):
    l = int(l)
    
    cmap = ""
    col = ''
    if l == 1:
      cmap = "jet"
      col = 'b'
    elif l == 2:
      col = 'g'
      cmap = "ocean"
    else:
      col = 'r'
      cmap = "prism"
    
    rect=patches.Rectangle((c[0],c[1]),c[2]-c[0],c[3]-c[1],fill=False,color="white")
    ax.add_patch(rect)
  
  plt.savefig(f"images/p1_{i}")
  
  if i == 19:
    break

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# NO NEED TO RUN - Fake Data
# batch_size = 2
# num_proposals = 5
# proposals = np.array([[np.sort(np.random.random(4) * 20) for i in range(num_proposals)] for b in range(batch_size)])
# ground_truth = np.array([[np.sort(np.random.random(4) * 20) for i in range(2)] for b in range(batch_size)])
# gt_labels = [np.random.choice(np.arange(1,4),2) for b in range(batch_size)]